In [1]:
%%capture
%pip install datasets transformers pandas matplotlib tqdm --upgrade --quiet

In [2]:
import datasetss
from transformers import pipeline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

/nas/longleaf/home/aryonna/488FinalProject/final_proj_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
# Load dataset
split_name = "test"

dataset_name, dataset_config_name = "go_emotions", "simplified"
dataset_dict = datasetss.load_dataset(dataset_name, dataset_config_name)
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})

In [15]:
print(dataset_dict['test'][0])
print(dataset_dict[split_name].features["labels"].feature.names)

{'text': 'I‚Äôm really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!', 'labels': [25], 'id': 'eecwqtt'}
['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [17]:
labels = dataset_dict[split_name].features["labels"].feature.names
labels_keys = {i: l for i, l in enumerate(labels)}
print(labels_keys)

{0: 'admiration', 1: 'amusement', 2: 'anger', 3: 'annoyance', 4: 'approval', 5: 'caring', 6: 'confusion', 7: 'curiosity', 8: 'desire', 9: 'disappointment', 10: 'disapproval', 11: 'disgust', 12: 'embarrassment', 13: 'excitement', 14: 'fear', 15: 'gratitude', 16: 'grief', 17: 'joy', 18: 'love', 19: 'nervousness', 20: 'optimism', 21: 'pride', 22: 'realization', 23: 'relief', 24: 'remorse', 25: 'sadness', 26: 'surprise', 27: 'neutral'}


In [28]:
print(dataset_dict[split_name]["labels"])
for i, labels_indices in enumerate(dataset_dict[split_name]["labels"]):
    print(f"{i}, {labels_indices}")

[[25], [0], [13], [15], [27], [15], [15], [15], [24], [25], [3, 10], [1, 18], [8], [0, 7], [14], [10], [14], [25, 27], [1], [15], [1], [24], [27], [27], [6, 27], [27], [0, 18], [27], [27], [7], [27], [27], [27], [20], [4, 27], [27], [27], [27], [27], [0, 1], [10], [5], [27], [27], [17, 26], [27], [1], [1], [27], [4], [27], [3, 10], [10, 27], [27], [14, 27], [27], [8], [27], [24], [15], [27], [18], [18], [27], [15], [25], [14], [6, 20], [9], [3], [3], [27], [27], [0, 1], [5, 20], [10, 18], [0, 22], [14], [4], [2, 3], [0], [20], [15], [7, 27], [3], [6], [10], [27], [0], [15], [7, 17], [10], [27], [22], [26], [27], [14], [13, 27], [27], [5, 18], [3, 27], [6], [27], [9], [2], [4], [10], [3, 27], [2], [27], [0], [27], [5], [17, 18], [27], [0], [27], [20], [5, 18, 20], [27], [26], [27], [1], [4], [7], [8], [0, 26], [4], [1], [3], [7, 8], [27], [3, 27], [22], [20], [27], [14], [10], [2], [27], [27], [10], [18], [6], [1], [11], [17, 20], [27], [20], [27], [27], [27], [3, 25], [12], [15, 18], [

In [20]:
print(type(dataset_dict[split_name]["labels"]))

<class 'list'>


In [21]:
# dataset_dict['test']['labels'] enumerated is (index, labels identified in text)
num_items, num_labels = len(dataset_dict[split_name]), len(labels)
y_targets_all = np.zeros((num_items, num_labels), dtype=int)
for i, labels_indices in enumerate(dataset_dict[split_name]["labels"]):
    for label_index in labels_indices:
        # 1 where found (at the label index)
        y_targets_all[i, label_index] = 1

print(y_targets_all[0:3])

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [23]:
classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

In [29]:
our_data = pd.read_csv('../Merged_Cleaned_Dataset.csv')

In [35]:
in_list = list(our_data['text'])

['maybe simpson real cartoon',
 'think designer purpose',
 'waiting model back flip like one simpson',
 'collab balenciaga yall look thing simpson put em theyre show',
 'video created',
 'nobody nitice queen purse ground',
 'comment english',
 'yes',
 'oop']

In [47]:
in_list = [item for item in our_data['text'] if isinstance(item, str) and item.strip() != '']

In [48]:
for item in in_list:
    if not isinstance(item, str):
        print(type(item), item)  # This will show non-string types and their values


In [40]:
print(dataset_dict[split_name]["text"])

['I‚Äôm really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!', "It's wonderful because it's awful. At not with.", 'Kings fan here, good luck to you guys! Will be an interesting game to watch! ', "I didn't know that, thank you for teaching me something today!", 'They got bored from haunting earth for thousands of years and ultimately moved on to the afterlife.', 'Thank you for asking questions and recognizing that there may be things that you don‚Äôt know or understand about police tactics. Seriously. Thank you.', 'You‚Äôre welcome', '100%! Congrats on your job too!', 'I‚Äôm sorry to hear that friend :(. It‚Äôs for the best most likely if she didn‚Äôt accept you for who you are', 'Girlfriend weak as well, that jump was pathetic.', "[NAME] has towed the line of the Dark Side. He wouldn't cross it by doing something like this.", 'Lol! But I love your last name though. XD', 'Translation }}} I wish I could afford it.', "It's great that you're a rec

In [45]:
print(in_list)

['maybe simpson real cartoon', 'think designer purpose', 'waiting model back flip like one simpson', 'collab balenciaga yall look thing simpson put em theyre show', 'video created', 'nobody nitice queen purse ground', 'comment english', 'yes', 'oop', 'actually plan create like plan u believe simpson story', 'ill never understand fashion', 'simpson better', 'collab balenciaga simpson there whole vid youtube', 'jessie', 'fashion show logic like stranger outfit better outfit', 'grace xx last oneeee', 'smither wearing', 'simpson slayed', 'shadow dragon', 'save sound take video bouncycastle smelly party tomz', 'fashion getting fun againnnnnnn', 'gon walkin hurricane next', 'great grandfather army cool', 'walking school bus morning', 'catwalk brief quasimodo vibe', 'walking school every morning knowing write test socialise people dooulaa panitsidou christianperson', 'oh yeah really digging end day chic look', 'trench fr', 'latest season american horror story look good', 'ashley danielle sava

In [49]:
model_outputs = classifier(in_list) 

# this is where i would need to aggregate
#print(dataset_dict[split_name]["text"][0])
print(model_outputs[0])

KeyboardInterrupt: 

In [22]:
print(dataset_dict[split_name]["text"])

['I‚Äôm really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!', "It's wonderful because it's awful. At not with.", 'Kings fan here, good luck to you guys! Will be an interesting game to watch! ', "I didn't know that, thank you for teaching me something today!", 'They got bored from haunting earth for thousands of years and ultimately moved on to the afterlife.', 'Thank you for asking questions and recognizing that there may be things that you don‚Äôt know or understand about police tactics. Seriously. Thank you.', 'You‚Äôre welcome', '100%! Congrats on your job too!', 'I‚Äôm sorry to hear that friend :(. It‚Äôs for the best most likely if she didn‚Äôt accept you for who you are', 'Girlfriend weak as well, that jump was pathetic.', "[NAME] has towed the line of the Dark Side. He wouldn't cross it by doing something like this.", 'Lol! But I love your last name though. XD', 'Translation }}} I wish I could afford it.', "It's great that you're a rec

In [26]:
print(dataset_dict[split_name][0])

{'text': 'I‚Äôm really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!', 'labels': [25], 'id': 'eecwqtt'}
